In [1]:
#keras imports
import keras
from keras.datasets import mnist
from keras.models import Sequential,Model
from keras.layers import Input, Dense, Dropout, Flatten,merge
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D,GlobalMaxPooling2D,GlobalAveragePooling2D
from keras.layers.merge import Concatenate, concatenate
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
#other imports
import numpy as np
import json
import sklearn.model_selection as selection
from sklearn.preprocessing import MinMaxScaler
import keras.preprocessing.image as prep
import cv2
import shutil
import pandas as pd
from keras.optimizers import Adam
import os
from imgaug import augmenters as iaa
import imgaug as ia

Using TensorFlow backend.


In [3]:
img_rows, img_cols = 75, 75
input_shape = (img_rows, img_cols, 1)
trainfile = 'train.json'
testfile = 'test.json'
model_wights = 'model_wights.hdf5'

In [4]:
def float_or_na(value):
    return float(value if value != 'na' else 0)

In [5]:
def process_data(filename,inlcude_target=False):
    with open(filename) as f:
        data = json.load(f)
    X_ids = []
    X_band1 = np.zeros((len(data), img_rows*img_cols))
    X_band2 = np.zeros((len(data), img_rows*img_cols))
    X_angle = np.zeros((len(data),3))
    y = None
    if(inlcude_target):
        y = np.zeros(len(data))
    for i in range(len(data)):
        el = data[i]
        X_ids.append(el['id'])
        X_band1[i] = np.array(el['band_1'])
        X_band2[i] = np.array(el['band_2'])
        angle = float_or_na(el['inc_angle'])
        X_angle[i][0] = angle
        X_angle[i][1] = np.sin(angle)
        X_angle[i][2] = np.cos(angle)
        if(inlcude_target):
            y[i] = np.int(el['is_iceberg'])
    del data
    X_band1 = X_band1.reshape(X_band1.shape[0], img_rows, img_cols, 1)
    X_band2 = X_band2.reshape(X_band2.shape[0], img_rows, img_cols, 1)
    return X_ids, X_band1, X_band2, X_angle, y

In [38]:
def augment(item1,item2, aug_types=['rotate']):
    augmented = False
    im_shape = item1.shape
    x1 = item1
    x2 = item2
    if ('rotate' in aug_types and np.random.random() < 0.2):
        x1 = prep.random_rotation(x1, rg=90,
                                     row_axis=0, col_axis=1, channel_axis=2)
        x2 = prep.random_rotation(x2, rg=90,
                                     row_axis=0, col_axis=1, channel_axis=2)
        augmented = True

    if ('shift' in aug_types and np.random.random() < 0.2):
        width_rg = np.random.random() * 0.3
        height_rg = np.random.random() * 0.3
        x1 = prep.random_shift(x1, wrg=width_rg, hrg=height_rg,
                                  row_axis=0, col_axis=1, channel_axis=2)
        x2 = prep.random_shift(x2, wrg=width_rg, hrg=height_rg,
                                  row_axis=0, col_axis=1, channel_axis=2)
        augmented = True
            

    if ('vflip' in aug_types and np.random.random() < 0.5):
        x1 = cv2.flip(x1, 0).reshape(im_shape)
        x2 = cv2.flip(x2, 0).reshape(im_shape)
        augmented = True
    if ('hflip' in aug_types and np.random.random() < 0.5):
        x1 = cv2.flip(x1, 1).reshape(im_shape)
        x2 = cv2.flip(x2, 1).reshape(im_shape)
        augmented = True
    return ( augmented, x1, x2)

In [39]:
_, X_band1, X_band2, X_angle, y = process_data(trainfile,inlcude_target = True)

In [40]:
print(X_band1.shape)
print(X_band2.shape)
print(X_angle.shape)

(1604, 75, 75, 1)
(1604, 75, 75, 1)
(1604, 3)


In [42]:
count = X_band1.shape[0]
lens = []
for i in range(count):
    augmented, x1, x2 = augment(X_band1[i],X_band2[i],aug_types=['rotate','hflip','vflip','shift'])
    shape = (1,) + x1.shape
    if(augmented):
        X_band1 = np.vstack((X_band1,x1.reshape(shape)))
        X_band2 = np.vstack((X_band2,x2.reshape(shape)))
        X_angle = np.vstack((X_angle,X_angle[i].reshape((1,)+X_angle[i].shape)))
        y = np.hstack((y,y[i]))
print(X_band1.shape)
print(X_band2.shape)
print(X_angle.shape)

(2926, 75, 75, 1)
(2926, 75, 75, 1)
(2926, 3)


In [43]:
def conv_chain(input_tensor,conv_units, include_pooling = True, convcounts = 2):
    tensor  = input_tensor
    for i in range(convcounts):
        tensor = Conv2D(conv_units,(3, 3), activation='relu')(tensor)
    tensor = MaxPooling2D()(tensor)
    tensor = Dropout(0.2)(tensor)
    return tensor

In [44]:
#model params
bn_model = 0

band1_input = Input(shape=input_shape, name='band_1')
band1 = BatchNormalization(momentum=bn_model)(band1_input)
band1 = conv_chain(band1,16)
band1 = conv_chain(band1,32)
band1 = conv_chain(band1,64)
band1 = conv_chain(band1,128,convcounts=1)
band1 = GlobalMaxPooling2D()(band1)
band1 = Dropout(0.2)(band1)



band2_input = Input(shape=input_shape, name='band_2')
band2 = BatchNormalization(momentum=bn_model)(band2_input)
# band2 = conv_chain(band2,32)
band2 = conv_chain(band2,64)
band2 = conv_chain(band2,128)
band2 = GlobalMaxPooling2D()(band2)
band2 = Dropout(0.2)(band2)


angle_input = Input(shape=(X_angle.shape[1], ), name='angle')
classification = concatenate([band1, band2, BatchNormalization(momentum=bn_model)(angle_input)])
classification = Dense(128, activation = 'relu')(classification)
classification =  Dropout(0.2)(classification)
classification =  Dense(1, activation = 'sigmoid')(classification)

model = Model([band1_input, band2_input,angle_input], classification)
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
# shutil.rmtree('/Graph')
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
band_1 (InputLayer)              (None, 75, 75, 1)     0                                            
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 75, 75, 1)     4           band_1[0][0]                     
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 73, 73, 16)    160         batch_normalization_1[0][0]      
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 71, 71, 16)    2320        conv2d_1[0][0]                   
___________________________________________________________________________________________

In [45]:
def get_callbacks(filepath, patience=2,use_tensorboad=False):
    cb= []
    cb.append(EarlyStopping('val_loss', patience=patience, mode="min"))
    cb.append(ModelCheckpoint(filepath, save_best_only=True))
    if(use_tensorboad):
        cb.append(TensorBoard(log_dir='Graph', histogram_freq=0, write_graph=True, write_images=True))
        if(os.path.isdir("Graph")):
            shutil.rmtree('Graph')
    return cb

In [46]:
# !tensorboard --logdir=Graph

In [47]:
callbacks = get_callbacks(filepath=model_wights, patience=5)

In [48]:
model.fit([X_band1,X_band2,X_angle], y,
          batch_size=32,
          epochs=12, 
          validation_split=0.2,          
          callbacks=callbacks,
          verbose=1)

Train on 2340 samples, validate on 586 samples
Epoch 1/12
2340/2340 [==============================] - 280s - loss: 0.6077 - acc: 0.6590 - val_loss: 0.5605 - val_acc: 0.6928
Epoch 2/12
 448/2340 [====>.........................] - ETA: 209s - loss: 0.5479 - acc: 0.7121

KeyboardInterrupt: 

In [22]:
test_ids, X_test1, X_test2, X_angle_test, _ = process_data(testfile)

In [23]:
model.load_weights(filepath=model_wights)
prediction = model.predict([X_test1,X_test2,X_angle_test], verbose=1)

8424/8424 [==============================] - 336s   


In [24]:
submit_df = pd.DataFrame({'id': test_ids, 'is_iceberg': prediction.flatten()})
submit_df.to_csv("./submission.csv", index=False)